# CH 21

## TOC<a id='toc'></a>
* [Ch21 Notes](#ch21_notes)

### CH21 Notes <a id='ch20_notes'></a>
[toc](#toc)
###  Class Metaprogramming

* **class metaprogramming** is the art of creating classes at runtime
    - in python, classes are *first-class* objects (can be passed to and return by funcs, can be created at runtime ...)

* We usually think of *type* as a function, because we use it like one. But really `type` is a class.
* It behaves like a class that creates new classes, when invoked with three arguments: `MyClass = type('MyClass', (MySuperClass, MyMixin), {'x':42, 'x2': lambda self: self.x *2})`
    * argnames are: name, base, dict
    * the last one, dict, is a mapping of attributes names and attributes of the new class
* the novelty here is that instances of type() are classes

* named tuple follows a different approach:
    * they have a string called `_class_template`, which is basically the source code as a string
    * named tuple fills it blanks calling `_class_template.format(...)`
    * the resulting source code string is evaluated using the `exec` builtin
* His claim: avoid using exec and eval - they pose serious security risks.
    - Python offers sufficient introspection functionality to not need these most of the time

<hr>
<br>
class introspection:

    * `vars(x)` :  equivalent to `x.__dict__`
        - instance vars stored in dict
    * `dir(x)` : returns a dict of x's "attributes, its class's attributes, and recurseively upward"
        - something akin to all the things that can be accessed using the dot operator (or equiv getattribute).
        - recall methods are not stored in dict - in fact they are non-overriding descriptors (so they are in super class dict)
<hr>
<br>

In [1]:
class MyClass:
    class_var = 3
    
    def __init__(self):
        self.inst_var = 4
        
    def spit_vars(self):
        return( self.class_var, self.inst_var)
    
    @property
    def var(self):
        return self.inst_var

In [2]:
obj = MyClass()

In [3]:
vars(obj)

{'inst_var': 4}

In [4]:
vars(MyClass)

mappingproxy({'__module__': '__main__',
              'class_var': 3,
              '__init__': <function __main__.MyClass.__init__(self)>,
              'spit_vars': <function __main__.MyClass.spit_vars(self)>,
              'var': <property at 0x2a8bc582db8>,
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

In [18]:
vars(MyClass.__init__)

{}

In [21]:
type(type(MyClass.__init__))

type

In [23]:
help(type(MyClass.__init__))

Help on class function in module builtins:

class function(object)
 |  function(code, globals, name=None, argdefs=None, closure=None)
 |  
 |  Create a function object.
 |  
 |  code
 |    a code object
 |  globals
 |    the globals dictionary
 |  name
 |    a string that overrides the name from the code object
 |  argdefs
 |    a tuple that specifies the default argument values
 |  closure
 |    a tuple that supplies the bindings for free variables
 |  
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __get__(self, instance, owner, /)
 |      Return an attribute of instance, which is of type owner.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  ---------------------

In [8]:
MyClass.__dict__['__dict__']

<attribute '__dict__' of 'MyClass' objects>

In [9]:
type(MyClass.__dict__['__dict__'])

getset_descriptor

In [14]:
help(MyClass.__dict__['__dict__'])

Help on getset descriptor __main__.MyClass.__dict__:

__dict__
    dictionary for instance variables (if defined)



In [15]:
vars(obj)

{'inst_var': 4}

In [16]:
dir(obj)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'class_var',
 'inst_var',
 'spit_vars',
 'var']

# A Class Decorator For Custimizing Descriptors
* class decorator gets a class object and returns the same class or a modified one
* class decorators came after function decorators
* can use class decorator to set descriptor storage names to something reasonable. example code
```
def entity(cls):
    for key, attr in cls.__dict__.items():
        if isinstance(attr, Validated):
            type_name = type(attr).__name__
            attr.storage_name = '_{}#{}'.format(type_name, key)
    return cls
```
* class decorators are a simpler way of doing things that previously required a metaclass: customizing a class the moment it's created.
* drawback: they act only on the class where they are directly applied. This means subclasses of decorated class may or may not inherit the changes, depending on what these are.

# What happens when: import time versus runtime
* At import time, the interpreter parses the source coude of a .py module in one pass from top to bottom, and generates bytecode to be executed.
    - (from the internet) **bytecode** is computer object code that is processed by a program, usually referred to as a virtual machine, rather than by the real computer machine, the hardware processor.
        - the python interpreter is an implementation of a virtual machine
    - this is when syntax errors may occur
    - if there is up to date .pyc file avaialble in local `__pycache__`, those steps are skipped bc bytecode is ready to run
* Although compiling is definitely an import-time activity, other things may happend at that time
* in particular, `import` statement runs all *top-level code* of the imported module when it is imported for the first time
    - a def statement on the top level of a module causes interpreter to compile function body and bind it to its global name (but doesnt execute function)
    - for classes it is different: intepreter executed the body of every class, even classes nested in other classes.
        * this means attributes and methods of class are defined
        * class object itself it suilt
    - and it can do othet things that are more typical of 'runtime' like for example connect to a database (not a good idea)

In [32]:
import dis

def hello_world():
    print('Hello World!')

In [33]:
dis.dis(hello_world)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('Hello World!')
              4 CALL_FUNCTION            1
              6 POP_TOP
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


# Metaclasses 101

* a metaclass is a class factory, except that instead of a function, a metaclass is written as a class
    - in pyhton, classes are objects - so they must be instances of something. By default, they are instances of `type`
    - to avoid infinite regress, type is itself an instance of type (wtf??)
* Note that this is not saying the a class like `LinteItem` inherits from type - it is an instance of type. `LineItem` actually inherits from object.
* the classes `object` and `type` have a unique relationship: 
    - object is an instance of type
    - type is a subclass ob object
    - this relationship is magic - it cannot be expressed in Python because either class would have to exist before the other could de defined.
    * the fact the type is an instance of itself is also magic.

* other meta classes in the standard library: ABCMeta and Enum
* Every class is of instance `type` either directly or indirectly, but only metaclasses are also subclasses of type.
    - so metaclass inherits from type the power to construct classes

* MAIN POINT: all classes are instances of `type`, but metaclasses are also subclasses of `type` - so they act as class factories.

In [ ]:
print('<[100]> evalsupport module start')

def deco_alpha(cls):
 print('<[200]> deco_alpha')
 def inner_1(self):
 print('<[300]> deco_alpha:inner_1')
 cls.method_y = inner_1
 return cls


# BEGIN META_ALEPH
class MetaAleph(type):
 print('<[400]> MetaAleph body')
 def __init__(cls, name, bases, dic):
 print('<[500]> MetaAleph.__init__')
 def inner_2(self):
 print('<[600]> MetaAleph.__init__:inner_2')
 cls.method_z = inner_2
# END META_ALEPH

print('<[700]> evalsupport module end')

* the metaclass' init method gets called when importing a class - this is how class (instance of meta) gets created.

* metaclass init signature: `def __init__(self, name, base, dic):`

* When coding a metaclass, it's conventional to replace `self` with `cls` in the init method. This makes it clear that the instance under construction is a class. (but it is still an isntance)

* COMMENT: It appears that the meta class init overrides definitions in class that inherits from it. i.e if both write to the same attribute, metaclass wins. 

* can also modify metaclasses `__new__`, but that is almost never needed

example metaclass:
```
class EntityMeta(type):
    def __init__(cls, name, bases, attr_dict):
       super().__init__(name, bases, attr_dict)
       for key, attr in attr_dict.items():
           if isinstrance(attr, Validate):
               type_name = type(attr, Validated):
                   type_name = type(attr).__name__
                   attr.storage_name = '_{}#{}'.format(type_name, key)
```

* often, along with a metaclass, a convenience class whose metaclass is set to defined metaclass is provided. Now users can simply inherit from that.
    - this new class is an instance of the meta - does not inherit from it

# The Metaclass __prepare__ Special Method
* since python 3
* both the type constructor and the __new__ and __init__ methods of metaclasses receive the body of the class evaluated as a mpping of names to attributes
    - by default, the mapping is a dict - so order is lost
* `__prepare__` special method is relevant only in metaclasses, and it must be a class method.
    - it is invoked by the interpreter before the `__new__` method in the metaclass
    - it creates the mapping that will be filled with the attributes from the class body
    - signature: `def __prepare__(cls, name, bases):`
    - must return a mapping, which will be received as the last argument of the new and init when metaclass builds a new class.
    - typically all people use it for is to `return collection.OrderedDict()`

* Basically, the way I understand is that when creating a class:
    - prepare is called first; you get an mapping instance
    - read the class definition, and add attributes to map
    - then call new/init and pass that filled out map to them

In the real world, metaclasses are used in frameworks and libraries to perform:
    - attribute validation
    - applying decorators to many methods at once
    - object serialization or data conversion
    - object-relational mapping
    - object-based persistency
    - dynamic translation of class strcutures from other languages

# Classes as objects
* `__mro__` - method resolution order
* `__class__`, and `__name__`
* `cls.__bases__` - the tuple of bases
* `cls.__qualname__` - new in py3.3 holding the qualified name - which is a doted path from global scop to the class definition
* `cls.__subclasses__` - returns a list of immediate sublcasses
    - implementation uses weak reference to avoid circular references (because bases contain strong references to sueprclasses)
* `cls.mro()` - intepreter calls this when building a class to obtain tuple in `__mro__`.
    - metaclass can overrid to customize mro at construction

Same admonishon applies: "don't define custom ABCs (or metaclasses) in production code... if you feel the urge to do, I'd bet it's likely to be a case of ''all problems look like a nail" syndrom for somebody whi just got a shiny new hammer - you (and future maintainers of your code) will be much happier sticking with straightforward and simple code, eschewing such depths."